In [1]:
import numpy as np
np.random.seed(2016)
import os
import glob
import cv2
import datetime
import time
from sklearn.cross_validation import KFold
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, MaxoutDense
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, RemoteMonitor
from keras.utils import np_utils
from keras.regularizers import l2
from sklearn.metrics import log_loss

Using Theano backend.
Using gpu device 0: GRID K520 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5005)


In [2]:
img_rows, img_cols = 128, 128
nfolds = 10

def get_im_cv2(path, img_rows, img_cols):
    img = cv2.imread(path, 0)
    resized = cv2.resize(img, (img_cols, img_rows), cv2.INTER_LINEAR)
    return resized


def load_train(img_rows, img_cols):
    X_train = np.array()
    X_train_id = []
    mask_train = []
    start_time = time.time()

    print('Read train images')
    files = glob.glob("train/*[0-9].tif")
    for fl in files:
        flbase = os.path.basename(fl)
        img = get_im_cv2(fl, img_rows, img_cols)
        X_train.append(img)
        X_train_id.append(flbase[:-4])
        mask_path = "train/" + flbase[:-4] + "_mask.tif"
        mask = get_im_cv2(mask_path, img_rows, img_cols)
        mask_train.append(mask)

    print('Read train data time: {} seconds'.format(round(time.time() - start_time, 2)))
    return np.array(X_train), np.array(mask_train), np.array(X_train_id)


def load_test(img_rows, img_cols):
    print('Read test images')
    files = glob.glob("test/*[0-9].tif")
    X_test = []
    X_test_id = []
    total = 0
    start_time = time.time()
    for fl in files:
        flbase = os.path.basename(fl)
        img = get_im_cv2(fl, img_rows, img_cols)
        X_test.append(img)
        X_test_id.append(flbase[:-4])
        total += 1

    print('Read test data time: {} seconds'.format(round(time.time() - start_time, 2)))
    return np.array(X_test), np.array(X_test_id)


def rle_encode(img, order='F'):
    bytes = img.reshape(img.shape[0] * img.shape[1], order=order)
    runs = []
    r = 0
    pos = 1
    for c in bytes:
        if c == 0:
            if r != 0:
                runs.append((pos, r))
                pos += r
                r = 0
            pos += 1
        else:
            r += 1

    if r != 0:
        runs.append((pos, r))
        pos += r

    z = ''
    for rr in runs:
        z += str(rr[0]) + ' ' + str(rr[1]) + ' '
    return z[:-1]


def find_best_mask():
    files = glob.glob(os.path.join("train", "*_mask.tif"))
    overall_mask = cv2.imread(files[0], cv2.IMREAD_GRAYSCALE)
    overall_mask.fill(0)
    overall_mask = overall_mask.astype(np.float32)

    for fl in files:
        mask = cv2.imread(fl, cv2.IMREAD_GRAYSCALE)
        overall_mask += mask
    overall_mask /= 255
    max_value = overall_mask.max()
    koeff = 0.5
    overall_mask[overall_mask < koeff * max_value] = 0
    overall_mask[overall_mask >= koeff * max_value] = 255
    overall_mask = overall_mask.astype(np.uint8)
    return overall_mask


def create_submission(predictions, test_id, info):
    sub_file = os.path.join('submission_' + info + '_' + str(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")) + '.csv')
    subm = open(sub_file, "w")
    mask = find_best_mask()
    encode = rle_encode(mask)
    subm.write("img,pixels\n")
    for i in range(len(test_id)):
        subm.write(str(test_id[i]) + ',')
        if predictions[i][1] > 0.5:
            subm.write(encode)
        subm.write('\n')
    subm.close()


def get_empty_mask_state(mask):
    out = []
    for i in range(len(mask)):
        if mask[i].sum() == 0:
            out.append(0)
        else:
            out.append(1)
    return np.array(out)


def read_and_normalize_train_data(img_rows, img_cols):
    train_data, train_target, train_id = load_train(img_rows, img_cols)
    train_data = np.array(train_data, dtype=np.uint8)
    train_target = np.array(train_target, dtype=np.uint8)
    train_data = train_data.reshape(train_data.shape[0], 1, img_rows, img_cols)
    # Convert to 0 or 1
    train_target = get_empty_mask_state(train_target)
    train_target = np_utils.to_categorical(train_target, 2)
    train_target = train_target.astype('uint8')
    train_data = train_data.astype('float32')
    train_data /= 255
    print('Train shape:', train_data.shape)
    print(train_data.shape[0], 'train samples')
    return train_data, train_target, train_id


def read_and_normalize_test_data(img_rows, img_cols):
    test_data, test_id = load_test(img_rows, img_cols)
    test_data = np.array(test_data, dtype=np.uint8)
    test_data = test_data.reshape(test_data.shape[0], 1, img_rows, img_cols)
    test_data = test_data.astype('float32')
    test_data /= 255
    print('Test shape:', test_data.shape)
    print(test_data.shape[0], 'test samples')
    return test_data, test_id


def merge_several_folds_mean(data, nfolds):
    a = np.array(data[0])
    for i in range(1, nfolds):
        a += np.array(data[i])
    a /= nfolds
    return a.tolist()


def create_model(img_rows, img_cols):
    model = Sequential()
    
    model.add(Dropout(0.25, input_shape=(1, img_rows, img_cols)))
    
    model.add(Convolution2D(32, 9, 9, border_mode='same', subsample=(2, 2), W_regularizer=l2(0.005)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    
    model.add(Convolution2D(64, 7, 7, border_mode='same', subsample=(2, 2), W_regularizer=l2(0.005)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    
    #model.add(Convolution2D(16, 3, 3, border_mode='same'))
    #model.add(Activation('relu'))
    #model.add(Convolution2D(16, 3, 3, border_mode='same'))
    #model.add(Activation('relu'))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.5))
#
    #model.add(Convolution2D(32, 3, 3, border_mode='same'))
    #model.add(Activation('relu'))
    #model.add(Convolution2D(32, 3, 3, border_mode='same'))
    #model.add(Activation('relu'))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.5))
    #
    model.add(Flatten())
    model.add(MaxoutDense(128, W_regularizer=l2(0.003)))
    model.add(Activation('relu'))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    opt = SGD(0.02, 0.95, decay = 0.003, nesterov=True)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=["accuracy"])
    return model


def get_validation_predictions(train_data, predictions_valid):
    pv = []
    for i in range(len(train_data)):
        pv.append(predictions_valid[i])
    return pv


def getPredScorePercent(train_target, predictions_valid):
    perc = 0
    for i in range(len(train_target)):
        pred = 1
        if predictions_valid[i][0] > 0.5:
            pred = 0
        real = 1
        if train_target[i][0] > 0.5:
            real = 0
        if real == pred:
            perc += 1
    perc /= len(train_target)
    return perc

In [3]:
train_data, train_target, train_id = read_and_normalize_train_data(img_rows, img_cols)
test_data, test_id = read_and_normalize_test_data(img_rows, img_cols)

TypeError: Required argument 'object' (pos 1) not found

In [ ]:
batch_size = 64
nb_epoch = 100
random_state = 51
yfull_train = dict()
yfull_test = []
kf = KFold(len(train_data), n_folds=nfolds, shuffle=True, random_state=random_state)
num_fold = 0
sum_score = 0
for train_index, test_index in kf:
    model = create_model(img_rows, img_cols)
    X_train, X_valid = train_data[train_index], train_data[test_index]
    Y_train, Y_valid = train_target[train_index], train_target[test_index]
    num_fold += 1
    print('Start KFold number {} from {}'.format(num_fold, nfolds))
    print('Split train: ', len(X_train), len(Y_train))
    print('Split valid: ', len(X_valid), len(Y_valid))
    remote = RemoteMonitor(root='http://localhost:9000')
    model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          shuffle=True, verbose=1, validation_data=(X_valid, Y_valid), callbacks=[remote])
    predictions_valid = model.predict(X_valid, batch_size=batch_size, verbose=1)
    score = log_loss(Y_valid, predictions_valid)
    print('Score log_loss: ', score)
    # Store valid predictions
    for i in range(len(test_index)):
        yfull_train[test_index[i]] = predictions_valid[i]
    # Store test predictions
    test_prediction = model.predict(test_data, batch_size=batch_size, verbose=2)
    yfull_test.append(test_prediction)

predictions_valid = get_validation_predictions(train_data, yfull_train)
score = log_loss(train_target, predictions_valid)
print("Log_loss train independent avg: ", score)
print('Final log_loss: {}, rows: {} cols: {} nfolds: {} epoch: {}'.format(score, img_rows, img_cols, nfolds, nb_epoch))
perc = getPredScorePercent(train_target, train_id, predictions_valid)
print('Percent success: {}'.format(perc))
info_string = 'loss_' + str(score) \
                + '_r_' + str(img_rows) \
                + '_c_' + str(img_cols) \
                + '_folds_' + str(nfolds) \
                + '_ep_' + str(nb_epoch)
test_res = merge_several_folds_mean(yfull_test, nfolds)
create_submission(test_res, test_id, info_string)

In [ ]:
batch_size = 32
nb_epoch = 50
model = create_model(img_rows, img_cols)
remote = RemoteMonitor(root='http://localhost:9000')
model.fit(train_data, train_target, batch_size=batch_size,
          nb_epoch=nb_epoch,shuffle=True, verbose=1,
          validation_data=(train_data[-500:], train_target[-500:]),
          callbacks=[remote])

In [30]:
set(map(lambda s: s.split('_')[0], train_id))

{'1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '4',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '5',
 '6',
 '7',
 '8',
 '9'}

In [7]:
np.array([])

array([], dtype=float64)